## Two-dimensional examples Neural networks and Manifolds

In [18]:
import numpy as np
from numpy.random import uniform, permutation

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD, RMSprop

from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure, output_notebook, show

In [17]:
output_notebook()

Loading BokehJS ...

### Defining the training data

Generate an artificial training data set with two classes that is not linearly separable. This will be obtained by sampling points uniformly in a square area and splitting the points according to an inner and outer region (this is the example from Andrej Karpathy's demo).

In [19]:
p1 = uniform(-0.3, 0.3, (80,2))
p2 = uniform(-1.0, 1.0, (120,2))
#only keep the points that are within the disk and split into two regions
p1 = p1[[np.dot(p, p) <= (0.3*0.3) for p in p1]]
p2 = p2[[np.dot(p, p) > (0.5*0.5) for p in p2]]

In [20]:
#plot points to check
p = figure(plot_width=400, plot_height=400)
p.circle(p1[:,0], p1[:,1], size=10, color="blue", alpha=0.5)
p.circle(p2[:,0], p2[:,1], size=10, color="red", alpha=0.5)
show(p)

Concatenate the points into one data set and assign the classification labels.

In [21]:
label1 = np.zeros((len(p1),))
label2 = np.ones((len(p2),))

points = np.vstack((p1, p2))
labels = np.concatenate([label1, label2])
labels = keras.utils.to_categorical(labels, num_classes=2)

#shuffle points to (hopefully) improve learning performance
shuffle = np.random.permutation(np.arange(len(points)))
points = points[shuffle,:]
labels = labels[shuffle,:]

In [38]:
#plot points to check
label_to_color = {0:'blue', 1:'red'}
colors = [label_to_color[l] for l in np.argmax(labels, axis=1)]
p = figure(plot_width=400, plot_height=400)
p.circle(points[:,0], points[:,1], size=10, color=colors, alpha=0.5)
show(p)

### Construct the neural network

The data set is not linearly separable in 2D, so we will need at least a layer with 3 units for the classification. Here again we will create a callback to store the history of the weights during learning.

In [39]:
class WeightsHistory(keras.callbacks.Callback):
    def __init__(self, model):
        self.model = model
        
    def on_train_begin(self, logs={}):
        self.weights = []

    def on_batch_end(self, batch, logs={}):
        self.weights.append(self.model.get_weights())

In [48]:
classifier_2d = Sequential()
classifier_2d.add(Dense(3, input_shape=(2,), activation='tanh'))
classifier_2d.add(Dense(2, activation='softmax'))
classifier_2d.compile(optimizer=SGD(lr=0.5), loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
weights_history = WeightsHistory(classifier_2d)
classifier_2d.fit(points, labels, batch_size=5, epochs=20, callbacks=[weights_history])

Epoch 1/20
163/163 [==============================] - 0s - loss: 0.7234 - acc: 0.4540         
Epoch 2/20
163/163 [==============================] - 0s - loss: 0.6445 - acc: 0.5583     
Epoch 3/20
163/163 [==============================] - 0s - loss: 0.5718 - acc: 0.6871     
Epoch 4/20
163/163 [==============================] - 0s - loss: 0.4788 - acc: 0.7853     
Epoch 5/20
163/163 [==============================] - 0s - loss: 0.3998 - acc: 0.8098     
Epoch 6/20
163/163 [==============================] - 0s - loss: 0.3666 - acc: 0.8589     
Epoch 7/20
163/163 [==============================] - 0s - loss: 0.3332 - acc: 0.8528     
Epoch 8/20
163/163 [==============================] - 0s - loss: 0.3308 - acc: 0.8773     
Epoch 9/20
163/163 [==============================] - 0s - loss: 0.2598 - acc: 0.9018     
Epoch 10/20
163/163 [==============================] - 0s - loss: 0.1715 - acc: 0.9693     
Epoch 11/20
163/163 [==============================] - 0s - loss: 0.1194 - acc: 0.975

As before, we precompute all the transformations so we can show the evolution during learning with an interactive plot in Bokeh. In this case we will show in a color plot how the classification progresses in the domain([-1,1],[-1,1]). This is similar to what Andrej Karpathy shows in the ConvNetJS demo. 

In [42]:
limits = (-1.0, 1.0)

grid_values = np.linspace(limits[0], limits[1], num=40)
grid_x, grid_y = np.meshgrid(grid_values, grid_values)

grid_pnts = np.asarray([[x,y] for x,y in zip (grid_x.reshape((grid_x.size,)), grid_y.reshape((grid_y.size,)))])

In [50]:
data_grid = {'labels': []}

for w in weights_history.weights:
    classifier_2d.set_weights(w)
    data_grid['labels'].append(np.argmax(classifier_2d.predict(grid_pnts), axis=1).reshape(grid_x.shape))

In [57]:
#apparently the data has to be stored in a field 'image'
#https://github.com/bokeh/bokeh/issues/5706
source_grid_current = ColumnDataSource(data=dict(image=[data_grid['labels'][0]]))
source_grid = ColumnDataSource(data=data_grid)

p = figure(plot_width=400, plot_height=400, x_range=limits, y_range=limits)
p.image(image='image', source=source_grid_current, 
        x=-1.0, y=-1.0, dw=2.0, dh=2.0, palette=['#1f77b4', '#d62728'])
p.circle(p1[:,0], p1[:,1], size=10, color="blue", alpha=0.5, line_color='black')
p.circle(p2[:,0], p2[:,1], size=10, color="red", alpha=0.5, line_color='black')

#note: "source.trigger('change')" should be replaced with "source.change.emit()" when using bokeh 0.12.6+
callback = CustomJS(args=dict(src=source_grid_current, 
                              src_grid=source_grid), code="""
                        var index = cb_obj.value
                        
                        var dat = src.data
                        var dat_grid = src_grid.data
                        dat['image'][0] = dat_grid['labels'][index]
                        
                        src.trigger('change')
                    """)

max_val = len(data_grid['labels'])-1                
slider = Slider(start=0, end=max_val, value=0, step=1, width=600)
slider.js_on_change('value', callback)

show(column(slider, p))

#layout = column(slider, p)

# update image on callback
#def update(index=0):
#    # update column data source
#    img.data_source.data['image'] = [data_grid['labels'][index]]
#    push_notebook()

#show(p, notebook_handle=True)